# Add libraries

In [9]:
from math import sqrt, log
import random



# Define some function

In [14]:
def get_n():
    return 600


def get_k():
    return 300

def r_t(n_t, s_t,n):
    c = 1
    s = min(1, s_t)
    return (c * log(n) / (n_t + 1)) + sqrt((c * log(n) * s) / (n_t + 1))


def desc_prices(delta):
    if delta > 0.618:
        return [1]

    active = []
    i = 1
    while True:
        item = delta * ((1 + delta) ** i)
        if item <= 1:
            active.append(item)
            i += 1
        else:
            break

    return active

def argmax(a_list):
    max_value = max(a_list)
    indices = [index for index, value in enumerate(a_list) if value == max_value]
    return random.choice(indices)

def random_gauss(mean, std):
    return random.gauss(mean, std)

def random_uniform(start,end):
    return random.uniform(start,end)

def shuffle(a_list):
    return random.shuffle(a_list)

def generate_random_c(n, k):
    delta = (k ** (-1 / 3)) * (log(n) ** (2 / 3))
    c = random_uniform(0.1, 0.3) / delta
    assert 0 < c < 1
    return c


# Single Ticket Pricing Strategy

In [20]:

class Ticket:
    def __init__(self, departure, destination, capacity, customers, c):
        self.departure = departure
        self.destination = destination
        self.capacity = capacity
        # number of potential buyers
        self.customers = customers
        self.c = c
        assert self.c <= 1
        self.delta = self.c * (self.capacity ** (-1 / 3)) * (log(self.customers) ** (2 / 3))
        assert self.delta <= 1

        # Discretization Price
        self.active_prices = desc_prices(self.delta)

        # len(index) == len(active_prices) 
        self.index = []

        # Averge sales rate at each price section
        self.sp = [0] * len(self.active_prices)

        # Number of items sold at each price section
        self.kt = [0] * len(self.active_prices)  

        # Number of rounds before t in which price has been chosen
        self.nt = [0] * len(self.active_prices)  
 
        # Quantity sold items
        self.total_sells = 0

        # Qunitity of sold itmes in bundles
        self.total_sell_bundle = 0

        self.available_seats = self.capacity - self.total_sells - self.total_sell_bundle

        # Total amount of revenue 
        self.revenue = 0

        # Inital value for propose_price is 1 (Highest possible Value)
        self.propose_price = 1

        # first choice of index is highest price
        self.act_index = -1

    def update_index(self):
        self.index = [
            self.active_prices[i]
            * ((self.customers * (min(1, self.sp[i]) + r_t(self.nt[i], self.sp[i],self.customers))))
            for i in range(len(self.active_prices))
        ]

    def sold(self):
        self.total_sells += 1
        self.revenue += self.propose_price
        self.kt[self.act_index] += 1
        self.sp[self.act_index] = self.kt[self.act_index] / min(
            1, self.nt[self.act_index]
        )
        self.update_available_seats()

    def sell_in_bundle(self):
        self.total_sell_bundle += 1
        self.update_available_seats()
    
    def update_available_seats(self):
        self.available_seats = self.capacity - self.total_sells - self.total_sell_bundle


    def offer_price(self):
        if self.capacity - self.total_sells - self.total_sell_bundle < 1:
            self.propose_price = 10 ** 34  # Some large value
        else:
            self.update_index()
            self.act_index = argmax(self.index)
            self.propose_price = self.active_prices[self.act_index]
            self.nt[self.act_index] += 1
            self.sp[self.act_index] = self.kt[self.act_index] / min(1, self.nt[self.act_index])


# Bundle Pricing Strategy

In [18]:

class Bundle():
    def __init__(self, departure, destination, customers, c, list_cities):
        self.departure = departure
        self.destination = destination
        self.customers = customers
        self.c = c
        assert self.c <= 1
        self.delta = [[0]] * len(list_cities)
        self.sp = [[0]] * len(list_cities)
        self.nt = [[0]] * len(list_cities)
        self.kt = [[0]] * len(list_cities)
        self.active_prices = [[]] * len(list_cities)
        self.list_cities = list(list_cities)  # EX. [[Tehran, Istanbul, NYC],[Tehran, Dubai, NYC]]
        self.index = [[]] * len(list_cities)
        self.sp = [[]] * len(list_cities)
        self.kt = [[]] * len(list_cities)
        self.nt = [[]] * len(list_cities)
        self.total_sells = [0] * len(list_cities)
        self.update_available_seats()
        self.propose_price = 1
        self.act_index = [[-1]] * len(list_cities)
        self.total_sells = [0] * len(list_cities)
        self.revenue = [0] * len(list_cities)
        self.history_sales = [[0]] * len(list_cities)  # History of sales
        self.failed_offers = [[0]] * len(list_cities)  # History of failed offers

    def update_available_seats(self):

        list_flight_seats = []  # all possible flight
        for bundle in range(len(self.list_cities)):
            seats = []
            for flight in range(len(self.list_cities[bundle]) - 1):
                flight_name = f"{self.list_cities[bundle][flight]}_{self.list_cities[bundle][flight + 1]}"
                seat = eval(f"{flight_name}.available_seats")
                seats.append(seat)
            list_flight_seats.append(min(seats))

        self.list_flight_seats = list_flight_seats

    def offer_price(self):
        #  Sell the minimum priced package
        self.update_available_seats()
        available_prices = [0] * len(self.list_flight_seats)
        for flight in range(len(self.list_flight_seats)):
            if self.list_flight_seats[flight] > 0:
                self.update_index()
                self.act_index[flight] = argmax(self.index[flight])
                available_prices[flight] = self.active_prices[flight][self.act_index[flight]]
            else:
                available_prices[flight] = 10 ** 34  # Some large value

        self.latest_flight_offer = available_prices.index(min(available_prices))
        self.propose_price = available_prices[self.latest_flight_offer]
        if self.propose_price <= 1:
            self.nt[self.latest_flight_offer][self.act_index[flight]] += 1
            self.sp[self.latest_flight_offer][self.act_index[flight]] = self.kt[flight][self.act_index[flight]] / max(1,self.nt[flight][self.act_index[flight]])


            # Add price to `failed_offers` if sold, we'll remove it
            self.failed_offers[self.latest_flight_offer].append(min(available_prices))

    def update_index(self):

        for flight in range(len(self.list_flight_seats)):
            if self.list_flight_seats[flight] <= 0:
                break
            self.delta[flight] = self.c * (self.list_flight_seats[flight] ** (-1 / 3)) * (log(self.customers) ** (2 / 3))
            self.active_prices[flight] = desc_prices(self.delta[flight])

            self.sp[flight] = [0] * len(self.active_prices[flight])
            self.nt[flight] = [0] * len(self.active_prices[flight])
            self.kt[flight] = [0] * len(self.active_prices[flight])

            for price_index in range(len(self.history_sales[flight])):
                for i in range(len(self.active_prices[flight]) - 1, -1, -1):
                    if self.history_sales[flight][price_index] > self.active_prices[flight][i]:
                        self.sp[flight][i] += 1
                        self.nt[flight][i] += 1
                        break

            for historic_data in range(len(self.failed_offers[flight])):
                for i in range(len(self.active_prices[flight]) - 1, -1, -1):
                    if self.failed_offers[flight][historic_data] > self.active_prices[flight][i]:
                        self.nt[flight][i] += 1
                        break

            # Update Kt
            self.kt[flight] = [self.sp[flight][k] / max(1, self.nt[flight][k]) for k in range(len(self.sp[flight]))]

            # Update Index
            self.index[flight] = [self.active_prices[flight][i] * self.customers * (
                        min(1, self.sp[flight][i]) + r_t(self.nt[flight][i], self.sp[flight][i],self.customers)) for i in
                                  range(len(self.active_prices[flight]))]

    def sold(self):
        # Delete successful Offer and add it to `history_sales`
        del self.failed_offers[self.latest_flight_offer][-1]
        self.history_sales.append(self.propose_price)
        self.revenue[self.latest_flight_offer] += self.propose_price
        self.total_sells[self.latest_flight_offer] +=1
        # Ask each flights to reserve
        for flight in range(len(self.list_cities[self.latest_flight_offer]) - 1):
            eval(f"{self.list_cities[self.latest_flight_offer][flight]}_{self.list_cities[self.latest_flight_offer][flight + 1]}.sell_in_bundle()")


# Design a Scenario

In [22]:

c = 0.1



airport_cities = ["Tehran", "Istanbul", "Dubai", "Berlin"]
flights = [[0,1], [0,2],[1,3],[2,3]] # [Tehran-Istanbul], [Tehran-Dubai], [Istnabul-Berlin], [Duba-Berlin]

# List of cities in Bundle
list_cities = [['Tehran', 'Istanbul', 'Berlin'],['Tehran', 'Dubai', 'Berlin']]
bundles =  [[[0,1,3],[0,2,3]]]


Tehran_Istanbul = Ticket(
        departure="Tehran",
        destination="Istanbul",
        capacity=400,
        customers=600,
        c=c,
    )
Tehran_Dubai = Ticket(
        departure="Tehran",
        destination="Dubai",
        capacity=300,
        customers=400,
        c=c,
    )
Dubai_Berlin = Ticket(
        departure="Dubai",
        destination="Berlin",
        capacity=300,
        customers=100,
        c=c,
    )
Istanbul_Berlin = Ticket(
        departure="Istanbul",
        destination="Berlin",
        capacity=600,
        customers=800,
        c=c,
    )


Tehran_Berlin = Bundle(
            departure="Tehran",
            destination="Berlin",
            customers=500,
            c=c,
            list_cities=list_cities)



# Create list of Buyers
customers_list = ["Tehran_Berlin","Tehran_Dubai","Tehran_Istanbul","Dubai_Berlin","Istanbul_Berlin"]



t = 0
while True:
    t += 1
#     route = random.choices(customers_list, weights=[0.3,0.1,0.3,0.1,0.2],k=1)[0]
    route = random.choice(customers_list)

    if route == "Tehran_Berlin":
        Tehran_Berlin.offer_price()
        if Tehran_Berlin.propose_price <= random_uniform(0.3,0.5):
            Tehran_Berlin.sold()
            
    elif route == "Tehran_Dubai":
        Tehran_Dubai.offer_price()
        if Tehran_Dubai.propose_price <= random_uniform(0.5,0.8):
            Tehran_Dubai.sold()

    elif route == "Tehran_Istanbul":
        Tehran_Istanbul.offer_price()
        if Tehran_Istanbul.propose_price <= random_uniform(0.4,0.6):
            Tehran_Istanbul.sold()
    
    elif route =="Dubai_Berlin":
        Dubai_Berlin.offer_price()
        if Dubai_Berlin.propose_price <= random_uniform(0.5,0.8):
            Dubai_Berlin.sold()
        
    elif route =="Istanbul_Berlin":
        Istanbul_Berlin.offer_price()
        if Istanbul_Berlin.propose_price <= random_uniform(0.3,0.2):
            Istanbul_Berlin.sold()
    
    if t == 19000:
        break

